In [1]:
from elasticsearch import Elasticsearch

In [2]:
es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic","iamaryan"),
    ca_certs="C:/elasticsearch-8.15.2/config/certs/http_ca.crt"
)
es.ping()

True

## Prepare the data

In [30]:
import pandas as pd

df = pd.read_csv("myntra_products_catalog.csv").loc[:500]
df.head()

,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor
0,10017413,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,Unisex,11745,7,"Black and grey printed medium trolley bag, sec...",Black
1,10016283,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Women,5810,7,Beige & Grey made to measure kurta with churid...,Beige
2,10009781,SPYKAR Women Pink Alexa Super Skinny Fit High-...,SPYKAR,Women,899,7,Pink coloured wash 5-pocket high-rise cropped ...,Pink
3,10015921,Raymond Men Blue Self-Design Single-Breasted B...,Raymond,Men,5599,5,Blue self-design bandhgala suitBlue self-desig...,Blue
4,10017833,Parx Men Brown & Off-White Slim Fit Printed Ca...,Parx,Men,759,5,"Brown and off-white printed casual shirt, has ...",White


In [31]:
df.isna().value_counts()

ProductID  ProductName  ProductBrand  Gender  Price (INR)  NumImages  Description  PrimaryColor
False      False        False         False   False        False      False        False           469
                                                                                   True             32
Name: count, dtype: int64

In [32]:
df.fillna("None", inplace=True)

## Convert the relevant field to Vector using BERT model

In [33]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

d:\June_Projects\REACH\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [34]:
df["DescriptionVector"] = df["Description"].apply(lambda x: model.encode(x))

In [35]:
df.head()

,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor,DescriptionVector
0,10017413,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,Unisex,11745,7,"Black and grey printed medium trolley bag, sec...",Black,"[0.027645713, -0.0026341556, -0.003588426, 0.0..."
1,10016283,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Women,5810,7,Beige & Grey made to measure kurta with churid...,Beige,"[-0.024660744, -0.028755462, -0.02033244, 0.03..."
2,10009781,SPYKAR Women Pink Alexa Super Skinny Fit High-...,SPYKAR,Women,899,7,Pink coloured wash 5-pocket high-rise cropped ...,Pink,"[-0.046943355, 0.08182783, 0.048335187, -0.000..."
3,10015921,Raymond Men Blue Self-Design Single-Breasted B...,Raymond,Men,5599,5,Blue self-design bandhgala suitBlue self-desig...,Blue,"[-0.015098742, -0.010285478, 0.009487344, -0.0..."
4,10017833,Parx Men Brown & Off-White Slim Fit Printed Ca...,Parx,Men,759,5,"Brown and off-white printed casual shirt, has ...",White,"[-0.017746514, 0.0062095993, 0.021813946, 0.02..."


In [36]:
es.ping()

True

## Create new index in ElasticSearch!

In [38]:
if es.indices.exists(index="all_products"):
    print("Index already exists")


Index already exists


In [37]:
from indexMapping import indexMapping

es.indices.create(index="all_products", mappings=indexMapping)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [all_products/KHjom6ZGR7myz7uxUxWbkQ] already exists')

## Ingest the data into index

In [39]:
record_list = df.to_dict("records")

In [40]:
record_list[0]

{'ProductID': 10017413,
 'ProductName': 'DKNY Unisex Black & Grey Printed Medium Trolley Bag',
 'ProductBrand': 'DKNY',
 'Gender': 'Unisex',
 'Price (INR)': 11745,
 'NumImages': 7,
 'Description': 'Black and grey printed medium trolley bag, secured with a TSA lockOne handle on the top and one on the side, has a trolley with a retractable handle on the top and four corner mounted inline skate wheelsOne main zip compartment, zip lining, two compression straps with click clasps, one zip compartment on the flap with three zip pocketsWarranty: 5 yearsWarranty provided by Brand Owner / Manufacturer',
 'PrimaryColor': ' Black',
 'DescriptionVector': array([ 2.76457127e-02, -2.63415556e-03, -3.58842593e-03,  5.13587482e-02,
         3.09660304e-02,  1.40507035e-02,  7.27053918e-03,  3.13871577e-02,
        -6.23788275e-02, -3.82880564e-03,  3.15213986e-02,  7.55473524e-02,
         2.12640106e-03,  4.64893691e-02,  5.07449023e-02, -1.71941034e-02,
         1.22891990e-02, -1.95682645e-02, -9.6

In [41]:
for record in record_list:
    try:
        es.index(index="all_products", document=record, id=record["ProductID"])
    except Exception as e:
        print(e)

In [42]:
es.count(index="all_products")

ObjectApiResponse({'count': 501, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

## Search the data

In [43]:
input_keyword = "jeans"
vector_of_input_keyword = model.encode(input_keyword)

query = {
    "field" : "DescriptionVector",
    "query_vector" : vector_of_input_keyword,
    "k" : 5,
    "num_candidates" : 500, 
}

res = es.knn_search(index="all_products", knn=query , source=["ProductName","Description"])
res["hits"]["hits"]

C:\Users\mrary\AppData\Local\Temp\ipykernel_19052\3639959087.py:11: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  res = es.knn_search(index="all_products", knn=query , source=["ProductName","Description"])


[{'_index': 'all_products',
  '_id': '10009685',
  '_score': 0.5503423,
  '_source': {'ProductName': 'SPYKAR Women Blue Alexa Super Skinny Fit High-Rise Clean Look Stretchable Cropped Jeans',
   'Description': 'Blue medium wash 5-pocket high-rise cropped jeans, clean look, light fade, has a button and zip closure, and waistband with belt loops'}},
 {'_index': 'all_products',
  '_id': '10009681',
  '_score': 0.5484132,
  '_source': {'ProductName': 'SPYKAR Women Blue Adora Skinny Fit Mid-Rise Clean Look Ankle-Length Stretchable Jeans',
   'Description': 'Blue dark wash 5-pocket mid-rise ankle-length jeans, clean look, light fade, has a button and zip closure, and waistband with belt loops'}},
 {'_index': 'all_products',
  '_id': '10009693',
  '_score': 0.5482862,
  '_source': {'ProductName': 'SPYKAR Women Blue Adora Skinny Fit Mid-Rise Low Distress Stretchable Ankle-Length Jeans',
   'Description': 'Blue light wash 5-pocket mid-rise ankle-length jeans, low distress, heavy fade, has a but